# **Summarization**

- 參考:<br>
https://huggingface.co/docs/transformers/task_summary#summarization<br>

## **方法一**

透過class pipeline.<br>
簡易版.<br>

In [1]:
from transformers import pipeline
from IPython.display import clear_output

summarizer1 = pipeline("summarization")
summarizer2 = pipeline("summarization", model="csebuetnlp/mT5_multilingual_XLSum", tokenizer="csebuetnlp/mT5_multilingual_XLSum")
clear_output()

ARTICLE_en = """ New York (CNN)When Liana Barrientos was 23 years old, she got married in Westchester County, New York.
A year later, she got married again in Westchester County, but to a different man and without divorcing her first husband.
Only 18 days after that marriage, she got hitched yet again. Then, Barrientos declared "I do" five more times, sometimes only within two weeks of each other.
In 2010, she married once more, this time in the Bronx. In an application for a marriage license, she stated it was her "first and only" marriage.
Barrientos, now 39, is facing two criminal counts of "offering a false instrument for filing in the first degree," referring to her false statements on the
2010 marriage license application, according to court documents.
Prosecutors said the marriages were part of an immigration scam.
On Friday, she pleaded not guilty at State Supreme Court in the Bronx, according to her attorney, Christopher Wright, who declined to comment further.
After leaving court, Barrientos was arrested and charged with theft of service and criminal trespass for allegedly sneaking into the New York subway through an emergency exit, said Detective
Annette Markowski, a police spokeswoman. In total, Barrientos has been married 10 times, with nine of her marriages occurring between 1999 and 2002.
All occurred either in Westchester County, Long Island, New Jersey or the Bronx. She is believed to still be married to four men, and at one time, she was married to eight men at once, prosecutors say.
Prosecutors said the immigration scam involved some of her husbands, who filed for permanent residence status shortly after the marriages.
Any divorces happened only after such filings were approved. It was unclear whether any of the men will be prosecuted.
The case was referred to the Bronx District Attorney\'s Office by Immigration and Customs Enforcement and the Department of Homeland Security\'s
Investigation Division. Seven of the men are from so-called "red-flagged" countries, including Egypt, Turkey, Georgia, Pakistan and Mali.
Her eighth husband, Rashid Rajput, was deported in 2006 to his native Pakistan after an investigation by the Joint Terrorism Task Force.
If convicted, Barrientos faces up to four years in prison.  Her next court appearance is scheduled for May 18.
"""
ARTICLE_zh='夏日炎炎，許多人為了節省電費會選擇吹電風扇或是將冷氣溫度調高。此外，老一輩也認為整晚吹冷氣睡覺，容易對身體造成危害。然而日本一名睡眠診所的院長卻指出，其實在炎熱的夏夜中，在適當的溫度下休息，才能有效緩解腦部的疲勞。他也提醒，睡覺時室溫最好保持在26度之下，否則自律神經為了排汗、降溫，不斷運作，大腦也無法好好休息。'

In [2]:
#英文
#比較不同模型的摘要差異
print(summarizer1(ARTICLE_en, max_length=130, min_length=30, do_sample=False))
print(summarizer2(ARTICLE_en, max_length=130, min_length=30, do_sample=False))

print('-'*30)

#繁中
print(summarizer1(ARTICLE_zh, max_length=130, min_length=30, do_sample=False)) #模型1是英文的摘要模型,因此出來的結果為亂碼.
print(summarizer2(ARTICLE_zh, max_length=130, min_length=30, do_sample=False))

[{'summary_text': ' Liana Barrientos, 39, is charged with two counts of "offering a false instrument for filing in the first degree" In total, she has been married 10 times, with nine of her marriages occurring between 1999 and 2002 . At one time, she was married to eight men at once, prosecutors say .'}]
[{'summary_text': 'A woman has pleaded not guilty to married 10 times in a row over alleged immigration scams in New York.'}]
------------------------------
[{'summary_text': " 炎 炎炎，   \xa0- \xa0: 牉: \xa0“I’ve never been happier.”’s been happier than ever.’�s. “”: “I've been happy to be happy’,” says a Chinese woman ."}]
[{'summary_text': '炎熱的夏日,許多人紛紛選擇在適當的溫度下睡覺,以緩解大腦疲勞。'}]


## **方法二**

詳細版.<br>

In [3]:
#載入訓練資料(未完成)

# from datasets import load_dataset

# # for available language names, see above
# src_lang = "english"
# tgt_lang = "bengali"

# ds = load_dataset(f"csebuetnlp/CrossSum", "{}-{}".format(src_lang, tgt_lang))

In [4]:
import re
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

WHITESPACE_HANDLER = lambda k: re.sub('\s+', ' ', re.sub('\n+', ' ', k.strip()))

model_name= 'csebuetnlp/mT5_multilingual_XLSum'
# model_name = "csebuetnlp/mT5_m2o_chinese_simplified_crossSum"
#此處模型選擇上
#可選擇配置對應的模型
#或是使用auto class自動尋找模型使用
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
clear_output()

In [5]:
#文章

#
article_text1 = '''夏日炎炎，許多人為了節省電費會選擇吹電風扇或是將冷氣溫度調高。此外，老一輩也認為整晚吹冷氣睡覺，容易對身體造成危害。然而日本一名睡眠診所的院長卻指出，其實在炎熱的夏夜中，在適當的溫度下休息，才能有效緩解腦部的疲勞。他也提醒，睡覺時室溫最好保持在26度之下，否則自律神經為了排汗、降溫，不斷運作，大腦也無法好好休息。'''
#
article_text2 = '根據《集英社》報導，東京疲勞睡眠診所院長梶本修身醫師表示，大腦在睡眠時活動力會下降，並讓腦部溫度降低約1度，以消除疲勞。然而在炎熱且難以入睡的環境下就寢，為了使腦部溫度下降，自律神經會不斷發出指令，讓身體提高呼氣頻率、促進排汗來散熱。原本是要讓大腦消除疲勞的睡眠過程，反而因為自律神經的過度運轉，導致疲勞累積，越睡越累。'

In [6]:
input_ids= tokenizer(
    [WHITESPACE_HANDLER(article_text1)],
    return_tensors="pt",
    padding="max_length",
    truncation=True,
    max_length=512
)["input_ids"]

output_ids1 = model.generate(
    input_ids=input_ids,
    min_length=10,
    max_length=200,
    no_repeat_ngram_size=2,
    num_beams=4
)[0]

summary1 = tokenizer.decode(
    output_ids1,
    skip_special_tokens=True,
    clean_up_tokenization_spaces=False
)

In [7]:
input_ids2 = tokenizer(
    [WHITESPACE_HANDLER(article_text2)],
    return_tensors="pt",
    padding="max_length",
    truncation=True,
    max_length=512
)["input_ids"]

output_ids2 = model.generate(
    input_ids=input_ids2,
    min_length=10,
    max_length=200,
    no_repeat_ngram_size=2,
    num_beams=4
)[0]

summary2 = tokenizer.decode(
    output_ids2,
    skip_special_tokens=True,
    clean_up_tokenization_spaces=False
)

In [8]:
print('文本1:')
print(summary1)
print('文本2:')
print(summary2)

文本1:
炎熱的夏日,許多人紛紛選擇在適當的溫度下睡覺。
文本2:
東京疲勞睡眠診所宣佈,在炎熱的環境下就寢,讓大腦恢復活動力。
